# One At the Time (OAT) Sensitivity analysis

This is the simplest case. Also called a _local_ Sensitivity analysis. One parameter is changed by keeping all the other constant and the difference in results is compared to the change. This allows to investigate how much results are affected by the specific change in the parameter. This is good for some types of analysis, but has some problems. Main issue is that the effect of a change in the parameter might be different when other parameters assume different values...so OAT can be misleading!  This problem can only be solved with a global sensitivity analysis (next notebook)

This notebook show how to perform a simple One A Time (OAT) sensitivity analysis for a example product system of a biobased product, and calculated sensitivity ratios for each parameter.

In [1]:
# Import brightway2.5 packages
import bw2calc as bc
import bw2data as bd
import bw_processing as bwp
import numpy as np
import pandas as pd
from scipy import stats
from lci_to_bw2 import * # import all the functions of this module

In [ ]:
bd.projects.set_current('advlca25') # Still working in the same project
bd.databases

Databases dictionary with 11 object(s):
	ALIGNED-biob-prod-dummy
	SAtestdb
	biosphere3
	carbon fiber
	ecoinvent 3.9.1 conseq
	exldb
	gsa_db
	sa_db
	sa_db_param
	testbiosphere
	testdb

We start by importing data about a fictional ("dummy") product system for a biobased product.

The product system includes different activities such as the production, use, and end of life of the biobased product.

In [3]:
# Import the dummy product system

# import data from csv
mydata = pd.read_csv('ALIGNED-LCI-biobased-product-dummy.csv', header = 0, sep = ",") # using csv file avoids encoding problem
mydata.head()

# keep only the columns needed
mydb = mydata[['Activity database','Activity code','Activity name','Activity unit','Activity type',
               'Exchange database','Exchange input','Exchange amount','Exchange unit','Exchange type',
               'Exchange uncertainty type','Exchange loc','Exchange scale','Exchange negative', 'Exchange minimum', 'Exchange maximum', 
               'Simapro name',	'Simapro unit', 'Simapro type']].copy()

mydb['Exchange uncertainty type'] = mydb['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers
# Note: to avoid having both nan and values in the uncertainty column I use zero as default

# Create dictionary in bw format and write database to disk. 
# Shut down all other notebooks using the same project before doing this
bw2_db = lci_to_bw2(mydb) # a function from the lci_to_bw2 module

# write database
bd.Database('ALIGNED-biob-prod-dummy').write(bw2_db)

# check what foreground activities are included
for act in bd.Database('ALIGNED-biob-prod-dummy'):
    print(act, act['code'])

Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 5/5 [00:00<00:00, 3255.44it/s]

Vacuuming database 


'Biobased-product-eol' (kilogram, None, None) c8301e73-d521-4a89-998b-30b7e7751011
'Biomass-growth' (kilogram, None, None) a7d34649-9c10-4423-bac3-ecab9b43b20c
'Biomass-processing' (kilogram, None, None) 403a5c32-c769-46fc-8b9a-74b8eb3c79d1
'Biobased-product-use' (year, None, None) f9eabf64-b899-40c0-9f9f-2009dbb0a0b2
'Biobased-product-manufacturing' (kilogram, None, None) a37d149a-6508-4563-8af6-e5a39b4176df


In [4]:
# More info 
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
myact._data

{'name': 'Biobased-product-use',
 'unit': 'year',
 'type': 'process',
 'database': 'ALIGNED-biob-prod-dummy',
 'code': 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2',
 'id': 23713}

We calculate a static climate impact score for the fictional biobased product.

In [5]:
# calculation of static LCA score
mymethod = ('IPCC 2013', 'climate change', 'global warming potential (GWP100)')
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
functional_unit = {myact: 1}
LCA = bc.LCA(functional_unit, mymethod)
LCA.lci()
LCA.lcia()
print("The static Global Warming impact score is", LCA.score, bd.methods[mymethod]['unit'])

The static Global Warming impact score is 121.67148771458245 kg CO2-Eq


We redo the same lca, but using datapackage, to be used for reference later on.

In [6]:
functional_unit, data_obj, _ = bd.prepare_lca_inputs(
    {myact: 1}, # we input a activity-based functional unit, the function returns a id-based functional unit.
    method=mymethod
)
LCA = bc.LCA(functional_unit, data_objs=data_obj)
LCA.lci()
LCA.lcia()
print("The static Global Warming impact score is", LCA.score, bd.methods[mymethod]['unit']) # here should get the same score as above.

static_lca_score = LCA.score # we save the static lca score for comparison later.

The static Global Warming impact score is 121.67148771458245 kg CO2-Eq


### Now perform sensitivity analysis

The procedure is in **two steps**: 

1) A simulation is performed where the initial parameter (value of each exchange) is increased by 10%. New results are calculated. 

2) Sensitivity rations are calculated for each parameter and then ranked. 


#### Step 1

All the scenarios are already saved in `ALIGNED-LCI-biobased-product-dummy.csv`.

In [7]:
# observe all scenarios, each scenario changes one parameter(change the value up by 10%).
dfs = pd.read_csv("ALIGNED-LCI-biobased-product-dummy.csv")
scenario_dfs = dfs.loc[:, "A":]
scenario_dfs

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R
0,1.1,1.00,1.00,1.0,1.0,1.00,1.00,1.0,1.00,1.00,1.00,1.0,1.0,1.0,1.00,1.0,1.00,1.0
1,0.5,0.55,0.50,0.5,0.5,0.50,0.50,0.5,0.50,0.50,0.50,0.5,0.5,0.5,0.50,0.5,0.50,0.5
2,0.5,0.50,0.55,0.5,0.5,0.50,0.50,0.5,0.50,0.50,0.50,0.5,0.5,0.5,0.50,0.5,0.50,0.5
3,-1.0,-1.00,-1.00,-1.1,-1.0,-1.00,-1.00,-1.0,-1.00,-1.00,-1.00,-1.0,-1.0,-1.0,-1.00,-1.0,-1.00,-1.0
4,1.0,1.00,1.00,1.0,1.1,1.00,1.00,1.0,1.00,1.00,1.00,1.0,1.0,1.0,1.00,1.0,1.00,1.0
5,0.5,0.50,0.50,0.5,0.5,0.55,0.50,0.5,0.50,0.50,0.50,0.5,0.5,0.5,0.50,0.5,0.50,0.5
6,0.5,0.50,0.50,0.5,0.5,0.50,0.55,0.5,0.50,0.50,0.50,0.5,0.5,0.5,0.50,0.5,0.50,0.5
7,1.0,1.00,1.00,1.0,1.0,1.00,1.00,1.1,1.00,1.00,1.00,1.0,1.0,1.0,1.00,1.0,1.00,1.0
8,0.5,0.50,0.50,0.5,0.5,0.50,0.50,0.5,0.55,0.50,0.50,0.5,0.5,0.5,0.50,0.5,0.50,0.5
9,0.5,0.50,0.50,0.5,0.5,0.50,0.50,0.5,0.50,0.55,0.50,0.5,0.5,0.5,0.50,0.5,0.50,0.5


In [8]:
# convert the scenario dataframe to tuples for creating datapackage.
scenario_data_tuple = list(scenario_dfs.itertuples(index=False, name=None))
for item in scenario_data_tuple:
    print(item) # each column is one scenario, each row is one exchange

(1.1, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0)
(0.5, 0.55, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5)
(0.5, 0.5, 0.55, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5)
(-1.0, -1.0, -1.0, -1.1, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0)
(1.0, 1.0, 1.0, 1.0, 1.1, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0)
(0.5, 0.5, 0.5, 0.5, 0.5, 0.55, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5)
(0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.55, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5)
(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.1, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0)
(0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.55, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5)
(0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.55, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5)
(0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.55, 0.5, 0.5,

In [9]:
# get ids of all activities, we use ids as indices to create datapackages.
from_ids = []
to_ids = []

for index, row in dfs.iterrows():
    from_id = bd.Database(row["Exchange database"]).get(row["Exchange input"]).id
    to_id = bd.Database(row["Activity database"]).get(row["Activity code"]).id
    from_ids.append(from_id)
    to_ids.append(to_id)

print(len(from_ids), len(to_ids)) # check the length of 'from_ids' and 'to_ids', should be equal to the number of exchanges.

18 18


In [10]:
# generate indices tuples for creating datapackage.
scenarios_indices = list(zip(from_ids, to_ids))
scenarios_indices

[(23710, 23710),
 (21150, 23710),
 (6204, 23710),
 (1171, 23710),
 (23711, 23711),
 (11211, 23711),
 (8405, 23711),
 (23712, 23712),
 (23710, 23712),
 (23711, 23712),
 (11211, 23712),
 (23713, 23713),
 (23712, 23713),
 (23714, 23713),
 (12877, 23713),
 (23714, 23714),
 (11211, 23714),
 (1171, 23714)]

In [11]:
# separate technosphere and biosphere and get flip sign array.
scenario_tech_data, scenario_bio_data = [], []
scenario_tech_indices, scenario_bio_indices = [], []
t_flip = []

for type, data, indices in zip(dfs["Exchange type"], scenario_data_tuple, scenarios_indices):
    if type !="biosphere":
        scenario_tech_data.append(data)
        scenario_tech_indices.append(indices)
        if indices[0] == indices[1]:
            t_flip.append(False)
        else:
            t_flip.append(True)
    else:
        scenario_bio_data.append(data)
        scenario_bio_indices.append(indices)


print(f"Technosphere length: {len(scenario_tech_data), len(scenario_tech_indices), len(t_flip)}") # check: should have the same length
print(f"Biosphere length: {len(scenario_bio_data), len(scenario_bio_indices)}") # check: should have the same length

Technosphere length: (16, 16, 16)
Biosphere length: (2, 2)


In [12]:
# create the datapackage
scenario_dp = bwp.create_datapackage(sequential=True)

scenario_dp.add_persistent_array(
    matrix='technosphere_matrix',
    indices_array=np.array(scenario_tech_indices, dtype=bwp.INDICES_DTYPE),
    data_array=np.array(scenario_tech_data),
    flip_array=np.array(t_flip),
)

scenario_dp.add_persistent_array(
    matrix='biosphere_matrix',
    indices_array=np.array(scenario_bio_indices, dtype=bwp.INDICES_DTYPE),
    data_array=np.array(scenario_bio_data),
)

In [13]:
# get the datapackages(without scenarios) from database
functional_unit, data_obj, _ = bd.prepare_lca_inputs(
    {myact: 1}, # we input a activity-based functional unit, the function returns a id-based functional unit.
    method=mymethod
)

In [14]:
# run lca
LCA = bc.LCA(functional_unit, data_objs=data_obj + [scenario_dp], use_arrays=True)
LCA.lci()
LCA.lcia()

In [15]:
# iterate through all scenarios to get lca score
OAT_values = []
for label in scenario_dfs.columns:
    OAT_values.append(LCA.score)
    print(f"initial value: {static_lca_score}")
    print(f"end valuee({label}): {LCA.score}")
    print("---")
    next(LCA)

initial value: 121.67148771458245
end valuee(A): 117.00918623103385
---
initial value: 121.67148771458245
end valuee(B): 129.1842314797095
---
initial value: 121.67148771458245
end valuee(C): 121.78727558088484
---
initial value: 121.67148771458245
end valuee(D): 119.17148771442945
---
initial value: 121.67148771458245
end valuee(E): 120.00530893803217
---
initial value: 121.67148771458245
end valuee(F): 121.72268607834765
---
initial value: 121.67148771458245
end valuee(G): 123.45308600454828
---
initial value: 121.67148771458245
end valuee(H): 115.24991952023989
---
initial value: 121.67148771458245
end valuee(I): 126.80001934616476
---
initial value: 121.67148771458245
end valuee(J): 123.50428436846643
---
initial value: 121.67148771458245
end valuee(K): 121.77388444226568
---
initial value: 121.67148771458245
end valuee(L): 110.610443376754
---
initial value: 121.67148771458245
end valuee(M): 128.73521272803808
---
initial value: 121.67148771458245
end valuee(N): 126.77388444226568

Let's look at the results. 

Initial and increased ('upper') parameter values and initial and end values of the Global Warming Impact (GWI) scores. 

In [16]:
par_name = list(zip(dfs["Activity code"], dfs["Exchange input"]))
par_values = dfs["Exchange amount"].to_list()
par_upper_values = [scenario_dfs.iloc[i, i] for i in range(len(scenario_dfs))]
score_values = [static_lca_score] * len(scenario_dfs)

sr = pd.DataFrame([par_name, par_values, par_upper_values, score_values, OAT_values], 
                  index = ['par_name','par_initial', 'par_upper', 'GWI_initial', 'GWI_end']).T
sr.head()

,par_name,par_initial,par_upper,GWI_initial,GWI_end
0,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, a7d3464...",1.0,1.1,121.671488,117.009186
1,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 97601e6...",0.5,0.55,121.671488,129.184231
2,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 8e1f53c...",0.5,0.55,121.671488,121.787276
3,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 349b29d...",-1.0,-1.1,121.671488,119.171488
4,"(403a5c32-c769-46fc-8b9a-74b8eb3c79d1, 403a5c3...",1.0,1.1,121.671488,120.005309


#### Step 2

**Sensitivity ratios** (SR) are calculated using the formula for discrete distributions, see equation (2) in Bisinella et al. (2016) 

_Bisinella, V., Conradsen, K., Christensen, T.H., Astrup, T.F., 2016. A global approach for sparse representation of uncertainty in Life Cycle Assessments of waste management systems. International Journal of Life Cycle Assessment._ https://doi.org/10.1007/s11367-015-1014-4

In [17]:
# calcualte sensitivity ratios
sr['SR'] = ((sr['GWI_end']-sr['GWI_initial'])/sr['GWI_initial']) / ((sr['par_upper']-sr['par_initial'])/sr['par_initial']) 

sr['SR_abs'] = abs(sr['SR'])

In [18]:
sr.head()

,par_name,par_initial,par_upper,GWI_initial,GWI_end,SR,SR_abs
0,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, a7d3464...",1.0,1.1,121.671488,117.009186,-0.383188,0.383188
1,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 97601e6...",0.5,0.55,121.671488,129.184231,0.617461,0.617461
2,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 8e1f53c...",0.5,0.55,121.671488,121.787276,0.009516,0.009516
3,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 349b29d...",-1.0,-1.1,121.671488,119.171488,-0.205471,0.205471
4,"(403a5c32-c769-46fc-8b9a-74b8eb3c79d1, 403a5c3...",1.0,1.1,121.671488,120.005309,-0.136941,0.136941


We can rank the parameters to see the most sensitive ones. I use the absolute value of SR in this case.

In [19]:
sr_sorted = sr.sort_values('SR_abs', ascending = False, ignore_index = True).copy() # important to re-index...
sr_sorted

,par_name,par_initial,par_upper,GWI_initial,GWI_end,SR,SR_abs
0,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, f9eabf6...",1.0,1.1,121.671488,110.610443,-0.909091,0.909091
1,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 97601e6...",0.5,0.55,121.671488,129.184231,0.617461,0.617461
2,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, a37d149...",50.0,55.0,121.671488,128.735213,0.580557,0.580557
3,"(a37d149a-6508-4563-8af6-e5a39b4176df, a37d149...",1.0,1.1,121.671488,115.24992,-0.527779,0.527779
4,"(a37d149a-6508-4563-8af6-e5a39b4176df, a7d3464...",0.5,0.55,121.671488,126.800019,0.421506,0.421506
5,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, c8301e7...",50.0,55.0,121.671488,126.773884,0.419358,0.419358
6,"(c8301e73-d521-4a89-998b-30b7e7751011, 349b29d...",1.0,1.1,121.671488,126.671488,0.410943,0.410943
7,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, a7d3464...",1.0,1.1,121.671488,117.009186,-0.383188,0.383188
8,"(c8301e73-d521-4a89-998b-30b7e7751011, c8301e7...",1.0,1.1,121.671488,117.032945,-0.381235,0.381235
9,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 349b29d...",-1.0,-1.1,121.671488,119.171488,-0.205471,0.205471


What are the most sensitive parameters in plain English?

In [20]:
# scroll through the table, identify the specific exchanges.
testing = []

for i in range(0, sr_sorted.shape[0]):
    
    par = sr_sorted.iloc[i,0] 
    
    for i in bd.Database('ALIGNED-biob-prod-dummy').get(par[0]).exchanges():
        if i['input'][1] == par[1]:
            print(i)
            testing.append(str(i))

Exchange: 1.0 year 'Biobased-product-use' (year, None, None) to 'Biobased-product-use' (year, None, None)>
Exchange: 0.5 kilogram 'nutrient supply from NPK (26-15-15) fertiliser' (kilogram, RER, None) to 'Biomass-growth' (kilogram, None, None)>
Exchange: 50.0 kilogram 'Biobased-product-manufacturing' (kilogram, None, None) to 'Biobased-product-use' (year, None, None)>
Exchange: 1.0 kilogram 'Biobased-product-manufacturing' (kilogram, None, None) to 'Biobased-product-manufacturing' (kilogram, None, None)>
Exchange: 0.5 kilogram 'Biomass-growth' (kilogram, None, None) to 'Biobased-product-manufacturing' (kilogram, None, None)>
Exchange: 50.0 kilogram 'Biobased-product-eol' (kilogram, None, None) to 'Biobased-product-use' (year, None, None)>
Exchange: 1.0 kilogram 'Carbon dioxide, fossil' (kilogram, None, ('air',)) to 'Biobased-product-eol' (kilogram, None, None)>
Exchange: 1.0 kilogram 'Biomass-growth' (kilogram, None, None) to 'Biomass-growth' (kilogram, None, None)>
Exchange: 1.0 kilog

In [21]:
sr_sorted['par_long-name'] = testing
sr_sorted

,par_name,par_initial,par_upper,GWI_initial,GWI_end,SR,SR_abs,par_long-name
0,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, f9eabf6...",1.0,1.1,121.671488,110.610443,-0.909091,0.909091,Exchange: 1.0 year 'Biobased-product-use' (yea...
1,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 97601e6...",0.5,0.55,121.671488,129.184231,0.617461,0.617461,Exchange: 0.5 kilogram 'nutrient supply from N...
2,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, a37d149...",50.0,55.0,121.671488,128.735213,0.580557,0.580557,Exchange: 50.0 kilogram 'Biobased-product-manu...
3,"(a37d149a-6508-4563-8af6-e5a39b4176df, a37d149...",1.0,1.1,121.671488,115.24992,-0.527779,0.527779,Exchange: 1.0 kilogram 'Biobased-product-manuf...
4,"(a37d149a-6508-4563-8af6-e5a39b4176df, a7d3464...",0.5,0.55,121.671488,126.800019,0.421506,0.421506,Exchange: 0.5 kilogram 'Biomass-growth' (kilog...
5,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, c8301e7...",50.0,55.0,121.671488,126.773884,0.419358,0.419358,Exchange: 50.0 kilogram 'Biobased-product-eol'...
6,"(c8301e73-d521-4a89-998b-30b7e7751011, 349b29d...",1.0,1.1,121.671488,126.671488,0.410943,0.410943,"Exchange: 1.0 kilogram 'Carbon dioxide, fossil..."
7,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, a7d3464...",1.0,1.1,121.671488,117.009186,-0.383188,0.383188,Exchange: 1.0 kilogram 'Biomass-growth' (kilog...
8,"(c8301e73-d521-4a89-998b-30b7e7751011, c8301e7...",1.0,1.1,121.671488,117.032945,-0.381235,0.381235,Exchange: 1.0 kilogram 'Biobased-product-eol' ...
9,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 349b29d...",-1.0,-1.1,121.671488,119.171488,-0.205471,0.205471,"Exchange: -1.0 kilogram 'Carbon dioxide, fossi..."
